In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tqdm
import seaborn as sns
sns.set_style("whitegrid")

## Hodgkins - Huxley Model
The model is defined as an object class.

In [ ]:
class HHModel():
    def __init__(self, dt = 0.01, iter = 10000):
        self.gK_max = 0.36 # max conductance of K channel
        self.gNa_max = 1.20 # max condutance of Na channel
        self.gl = 0.003 # Leaky channel conductance
        self.E_Na = 50 # Nernst potential of Na channel
        self.E_K = -77 # Nernst potential of K channel
        self.E_l = -54.387 # Nernst potential of leaky channel
        self.C = 0.01 # Membrane conductance
        self.v_monitor = [] # Monitor to append all the voltage values
        self.gNa_monitor = [] # Monitor to append all the Na conductance values
        self.gK_monitor = [] # Monitor to append all the K conductance values
        self.m_monitor = [] # Monitor to append all the values of gating variable m
        self.n_monitor = [] # Monitor to append all the values of gating variable n
        self.h_monitor = [] # Monitor to append all the values of gating variable h
        self.dt = dt
        self.iter = iter

    def reset_monitors(self):
    
        self.v_monitor = []
        self.gNa_monitor = []
        self.gK_monitor = []
        self.m_monitor = []
        self.n_monitor = []
        self.h_monitor = []

    def model(self, I_app: float):
        '''
        Computes the voltage using Hodgkin Huxeley model
        -------------------------------------------------
        Args:
            I_app: External current (micro amperes)
            dt: Step length for Euler method. Default = 0.01 ms
            iter: Number of iterations. Default = 10000
        Returns:
            v_monitor: voltage
            m: Gating variable
            n: Gating variable
            h: Gating variable
            gNa: Conductance of Na channel
            gK: Conductance of K channel
        --------------------------------------------------
        '''
        self.reset_monitors()
        v = -64.9964
        m = 0.0530
        h = 0.5960
        n = 0.3177
        dt = self.dt
        iter = self.iter
        # Dynamic relationships specified

        for i in range(iter):
            gNa = self.gNa_max * (m**3) * h
            gK = self.gK_max * n**4
            
            dv = (-gNa * (v - self.E_Na) - gK * (v - self.E_K) - self.gl *(v- self.E_l) + I_app)/self.C
            v = v + dt*dv
        
            alpha_m = 0.1 * (v + 40) / (1- np.exp(-(v + 40)/10))
            beta_m = 4 * np.exp(-0.0556 * (v + 65))
            
            alpha_h = 0.07 * (np.exp(-0.05 *(v + 65)))
            beta_h = 1/(1 + np.exp(-0.1 * (v + 35)))
            
            alpha_n = 0.01 * (v + 55)/(1 - np.exp(-(v + 55)/10))
            beta_n = 0.125 * np.exp(-(v + 65)/80)
        
            dm = alpha_m * (1 - m) - beta_m * m
            dn = alpha_n * (1 - n) - beta_n * n
            dh = alpha_h * (1 - h) - beta_h * h
        
            m = m + dt * dm
            n = n + dt * dn
            h = h + dt * dh
        
            self.v_monitor.append(v)
            self.m_monitor.append(m)
            self.n_monitor.append(n)
            self.h_monitor.append(h)
            self.gNa_monitor.append(gNa)
            self.gK_monitor.append(gK)
        
        return self.v_monitor, self.m_monitor, self.n_monitor, self.h_monitor, self.gNa_monitor, self.gK_monitor


## Simulation Interface
Two simulations are specified. One simulates the spike frequency profile of the model under the given conditions. This is used to define threshold currents for different regimes.
This is used to visualize the dynamic behaviour of the Hodgkins - Huxley Neuron in different regimes, which is the second simulation.

In [ ]:
#Calculating Threshold Currents
from tqdm import tqdm

# Peak Detection Method
def peaks(v_vals):
	threshold = 10
	npeaks = 0
	for i in range(len(v_vals)-1):
		if (v_vals[i] >= threshold) and (v_vals[i] > v_vals[i+1]) and (v_vals[i] > v_vals[i-1]):
			npeaks += 1
	return	npeaks



current_range = np.arange(0, 0.8, 0.001)

# Conditions for Frequency Profile Simulation
total_simulation_time = 500
dt = 0.01
iter = int(total_simulation_time/dt)
HH = HHModel(dt, iter) # dt is in ms
HH_new = HHModel(dt,iter)

peak_monitor= []
for current in tqdm(current_range):
    v_vals, m_vals, n_vals, h_vals, gNa_vals, gK_vals = HH_new.model(current)
    peak_monitor.append(peaks(v_vals))


In [ ]:
# Method to detect current thresholds
def current_thresholds(peak_monitor):
    return_list = []
    for i in range(len(peak_monitor)-1):        
    
        if peak_monitor[i+1]-peak_monitor[i] > 10:
            return_list.append(i)
        elif peak_monitor[i] > 0 and peak_monitor[i-1] == 0:
            return_list.append(i)
        elif peak_monitor[i+1]-peak_monitor[i] < -40:
            return_list.append(i)
    return return_list

current_limits = current_thresholds(peak_monitor)

# Spike Frequency profile plot 

plt.figure() 
plt.plot(current_range, peak_monitor, label="Spike Frequency")
plt.axvline(x=current_range[current_limits[0]], color="r", label="I1")
plt.axvline(x=current_range[current_limits[1]], color="g", label="I2")
plt.axvline(x=current_range[current_limits[2]], color="orange", label="I3")
plt.title("Frequency of Spiking vs Input Current")
plt.xlabel(f"Input Current ($\mu\text{A}/\text{mm}^2$)")


plt.ylabel("Frequency of Spiking")
plt.legend()
plt.savefig("Frequency_sweep.jpeg", dpi=900, bbox_inches="tight")
plt.show()

print("\nThe cutoff currents are as follows:")
print("I1 =", current_range[current_limits[0]], "microA/mm^2")
print("I2 =", current_range[current_limits[1]], "microA/mm^2")
print("I3 =", current_range[current_limits[2]], "microA/mm^2")


In [ ]:
# Conditions for simulating dynamic behaviour
total_simulation_time = 100
dt = 0.01
iter = int(total_simulation_time/dt)
HH = HHModel(dt, iter) # dt is in ms

In [ ]:
# Method to plot graphs of voltage, gating variables, and conductances against time.
def plot_relevant_graphs(model, I_inj):
    t = np.arange(0, model.iter) * model.dt
    
    v_vals, m_vals, n_vals, h_vals, gNa_vals, gK_vals = HH.model(I_inj)
    plt.figure()
    plt.plot(t, v_vals)
    string = f"Voltage variation vs time: I = {I_inj} $\mu\\text{{A}}/\\text{{mm}}^2$"
    plt.title(string)
    plt.xlabel("Time (ms)")
    plt.ylabel("Voltage (mV)")
    plt.savefig(f"{I_inj}_A.jpeg", dpi=900, bbox_inches='tight')
    
    plt.figure()
    plt.plot(t, m_vals)
    plt.plot(t, h_vals)
    plt.plot(t, n_vals)
    plt.legend(['m','h','n'])
    string = f"Gating Variable Probablilites vs time: I = {I_inj} $\mu\\text{{A}}/\\text{{mm}}^2$"
    plt.title(string)
    plt.xlabel("Time (ms)")
    plt.ylabel("Gating variable probabilities")
    plt.savefig(f"{I_inj}_B.jpeg", dpi=900, bbox_inches='tight')
    
    plt.figure()
    plt.plot(t, gNa_vals)
    plt.plot(t, gK_vals)
    plt.legend(['g_Na','g_K'])
    string = f"Conductance variation vs time: I = {I_inj} $\mu\\text{{A}}/\\text{{mm}}^2$"
    plt.title(string)
    plt.xlabel("Time (ms)")
    plt.ylabel("Conductance")
    plt.savefig(f"{I_inj}_C.jpeg", dpi=900, bbox_inches='tight')

    plt.show()

In [ ]:
for current in [0.01, 0.023, 0.0617, 0.467, 0.6, 2.0 ]:
    plot_relevant_graphs(HH, current)